# Derive Top N Portfolio

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore')     #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; else set it to False to turn off debugging mode '''
debug = True

In [64]:
import os
import sys
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from utils.modules.etl.load import sparkDBwls as sdb
from utils.modules.etl.transform import sparkCleanNRich as scne
from mining.modules.assets.etp import dailyTopN as topN
# from utils.modules.ml.timeseries import rollingstats as stats

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    topN = importlib.reload(topN)
    sdb = importlib.reload(sdb)
    scne = importlib.reload(scne)
#     stats= importlib.reload(stats)
    
__desc__ = "analyze crypto market capitalization time series data"
# clsSDB = sdb.SQLWorkLoads(desc=__desc__)
clsSCNR=scne.Transformer(desc=__desc__)
clsMPT =topN.WeightedPortfolio(desc=__desc__)
# clsStat=stats.RollingStats(desc=__desc__)
''' optional - if not specified class will use the default values '''
# prop_kwargs = {"WRITE_TO_TMP":True,   # necessary to emulate the etl dag
#               }
print("\nClass initialization and load complete!")

All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All functional DAILYTOPN-libraries in ETP-package of ASSETS-module imported successfully!
All functional SPARKDBWLS-libraries in LOAD-package of ETL-module imported successfully!
All functional SPARKCLEANNRICH-libraries in TRANSFORM-package of ETL-module imported successfully!


INFO:transform:########################################################
INFO:transform:sparkCleanNRich transform
DEBUG:transform:utils initialization for etl module package transform sparkCleanNRich done.
Start workloads: analyze crypto market capitalization time series data.
INFO:etp:########################################################
INFO:etp:dailyTopN Class
INFO:load:########################################################
INFO:load:sparkdbwls load
DEBUG:load:utils initialization for etl module package load sparkdbwls done.
Start workloads: analyze crypto market capitalization time series data.
INFO:transform:########################################################
INFO:transform:sparkCleanNRich transform
DEBUG:transform:utils initialization for etl module package transform sparkCleanNRich done.
Start workloads: analyze crypto market capitalization time series data.
INFO:load:########################################################
INFO:load:sparkNoSQLwls Class
DEBUG:load:utils

sparkNoSQLwls Class initialization complete
dailyTopN Class initialization complete

Class initialization and load complete!


## Read data from mcap_past
We apply a query to select assets with mcap > 1.0 million. Any missing values are imputed with the mean value.

In [66]:
_from_date = '2022-01-05'
_to_date = '2022-01-10'

# _query = "select * from warehouse.mcap_past "+\
#         f"where mcap_date between '{_from_date}' and '{_to_date}' "+\
#         f"and mcap_value > 1000000 and asset_name in "+\
#         "('brg_x','_crdn','avme','atri','ethix','hoge','xpx','wabi','dmg','mintme','chart')"
_query = "select * from warehouse.mcap_past "+\
        f"where mcap_date between '{_from_date}' and '{_to_date}' "+\
        f"and mcap_value > 1000000"

_kwargs = {
    "TABLENAME":'warehouse.mcap_past',
    "COLUMN":'mcap_date',
    "FROMDATETIME":_from_date,
    "TODATETIME":_to_date,
    "PARTITIONS":2,
    "AGGREGATE":'avg',
    "PIVCOLUMNS":['dxd','sofi','wsn','xmx','uqc','btr','unic','nex','noia',
                  'hanu','aca','bbs','xvs','pnd','shake','stpl','dtx','tethys',
                  'kyoko','boba','nlife','rare','eved','yfl','fkx','flixx',
                  'drk','meto','glide','shr','tetu','mft','cmerge','shmn','tronpad']
}

# print(clsSpark.dbSchema)
mcap_sdf = clsMPT.read_n_clean_mcap(query=_query,**_kwargs)
# mcap_sdf = clsROR.read_n_clean_mcap(**_kwargs)

print("Loaded %d rows and %d columns" % (mcap_sdf.count(),len(mcap_sdf.columns)))

DEBUG:load:@setter Spark PARTIONS set to: 2
DEBUG:load:@property Database dbConnURL set to: jdbc:postgresql://127.0.0.1:5432/tip


Wait a moment, retrieving data ...


DEBUG:load:loaded 10913 rows into pyspark dataframe                             
DEBUG:etp:function <read_n_clean_mcap> loaded 10913 rows
DEBUG:transform:Pivoting 1875 valid columns
DEBUG:transform:Transposing 10913 rows groupby MCAP_DATE to pivot with distinct values in ASSET_NAME and AVG aggregation on column(s): mcap_value
DEBUG:etp:utils etl transform sparkCleanNRich Pivot process returned 1876 columns and 6 rows
DEBUG:transform:function <impute_data> validated 1875 numeric columns listed in attr: column_subset


23/02/15 21:52:35 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB


23/02/15 21:52:46 WARN DAGScheduler: Broadcasting large task binary with size 6.9 MiB


DEBUG:transform:IMputer succeded with returning 6 rows                          
DEBUG:etp:function <impute_wrapper> retuned impute data on 1875 asset tickers and 6 market cap row
DEBUG:transform:NULL count completed for 1875 columns
DEBUG:etp:Check for NaN, Null, and Empty cells returned 1875 columns and 1 rows
ERROR:transform:function <unpivot_table> 
Syntax error at or near ',': extra input ','(line 1, pos 16176)

== SQL ==
stack(1875,'trava',trava,'dnt',dnt,'rfuel',rfuel,'luffy',luffy,'sfund',sfund,'orn',orn,'mng',mng,'bsw',bsw,'nftp',nftp,'isp',isp,'rdpx',rdpx,'shak',shak,'auction',auction,'dough',dough,'launch',launch,'scifi',scifi,'bns',bns,'spwn',spwn,'noia',noia,'amp',amp,'viper',viper,'watch',watch,'coin',coin,'ryo',ryo,'cook',cook,'sail',sail,'gse',gse,'lunr',lunr,'sngls',sngls,'signa',signa,'thn',thn,'mobic',mobic,'tri',tri,'sav3',sav3,'euler',euler,'vtho',vtho,'ftx',ftx,'farm',farm,'swp',swp,'ss',ss,'oax',oax,'udoo',udoo,'xido',xido,'pi',pi,'wagmi',wagmi,'port',port,'husl'

ERROR:etp:function <impute_wrapper> 'NoneType' object has no attribute 'count' 

DEBUG:etp:Traceback (most recent call last):
  File "/home/nuwan/workspace/rezaware/mining/modules/assets/etp/dailyTopN.py", line 410, in impute_wrapper
    if self._data.count() > 0:
AttributeError: 'NoneType' object has no attribute 'count'



[Error]function <unpivot_table> 
Syntax error at or near ',': extra input ','(line 1, pos 16176)

== SQL ==
stack(1875,'trava',trava,'dnt',dnt,'rfuel',rfuel,'luffy',luffy,'sfund',sfund,'orn',orn,'mng',mng,'bsw',bsw,'nftp',nftp,'isp',isp,'rdpx',rdpx,'shak',shak,'auction',auction,'dough',dough,'launch',launch,'scifi',scifi,'bns',bns,'spwn',spwn,'noia',noia,'amp',amp,'viper',viper,'watch',watch,'coin',coin,'ryo',ryo,'cook',cook,'sail',sail,'gse',gse,'lunr',lunr,'sngls',sngls,'signa',signa,'thn',thn,'mobic',mobic,'tri',tri,'sav3',sav3,'euler',euler,'vtho',vtho,'ftx',ftx,'farm',farm,'swp',swp,'ss',ss,'oax',oax,'udoo',udoo,'xido',xido,'pi',pi,'wagmi',wagmi,'port',port,'husl',husl,'don',don,'ilsi',ilsi,'pawth',pawth,'dag',dag,'qkc',qkc,'kart',kart,'xep',xep,'ndx',ndx,'rly',rly,'gny',gny,'cnns',cnns,'wasp',wasp,'unfi',unfi,'bc',bc,'ioi',ioi,'pny',pny,'tsx',tsx,'dfd',dfd,'pma',pma,'ugotchi',ugotchi,'prob',prob,'gyro',gyro,'edda',edda,'ddd',ddd,'ionc',ionc,'celt',celt,'crbn',crbn,'pbx',pbx,'grim

AttributeError: 'NoneType' object has no attribute 'count'

## Compute LogROR for all assets

In [16]:
kwargs={
    "PREVALCOLNAME":'mcap_prev_val',
    "DIFFCOLNAME":'mcap_diff',
    "LOGCOLNAME":'log_ror'
}
_mcap_log_ror, _log_col = clsMPT.get_log_ror(
    data=mcap_sdf,
    num_col_name="mcap_value",
    part_column ='asset_name',
    **kwargs,
)

_mcap_log_ror.filter(_mcap_log_ror.log_ror.isNotNull()).show(n=3)

+-------------------+----------+--------------------+--------------------+--------------------+--------------------+
|          mcap_date|asset_name|          mcap_value| mcap_value_prev_val|           mcap_diff|             log_ror|
+-------------------+----------+--------------------+--------------------+--------------------+--------------------+
|2022-01-06 00:00:00|     ibeur|137865091.0487160...|136889535.6501430...|975555.3985730000...|-0.00308399183067...|
|2022-01-06 00:00:00|     stake|48797307.35199010...|48796207.64411680...|1099.707873300000...|-9.98888795642457...|
|2022-01-06 00:00:00|  safemoon|1445470498.470450...|1448946116.878680...|-3475618.40823000...|0.001042791001318...|
+-------------------+----------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



## Weighted Portfolio

In [5]:
_cols={
    "NAMECOLUMN":'asset_name',
    "DATECOLUMN":'mcap_date',
    "RORCOLUMN" :_log_col,
    "MCAPCOLUMN":'mcap_value',
    "WEIGHTCOLUMN":'weights',
}
_l_exp_wts,_cols_dict=clsMPT.get_weighted_mpt(
    data=_mcap_log_ror,
    cols_dict=_cols,
#     date_col='mcap_date',
#     val_col='log_ror',
#     name_col='asset_name',
    topN=3,
    size=5,
    **_kwargs,
)
# print("Dates: %s" % str(_wr_dates))
# print("Data : row=%d columns=%d" % (_wr_data.count(),len(_wr_data.columns)))
_l_exp_wts

NameError: name '_log_col' is not defined

## Write MPT to MongoDB

* Collection name = "mpt."+date(YYYY-MM-DD)
* document structure: \_id, date, asset, mcap.value, mcap.weight, mcap.ror

In [20]:
_kwargs = {
    "DESTINDBNAME":'tip-daily-mpt',
    "COLLPREFIX" : 'mpt.for'
}
mpt_list_ = clsMPT.write_mpt_to_db(
    mpt_data=_l_exp_wts,
    cols_dict=_cols_dict,
    **kwargs,
)
print("Upsert %d documents" % len(mpt_list_))

Upsert 5 documents
